In [1]:
using ITensors, ITensorMPS
using LinearAlgebra
using Plots

N = 10
s = siteinds("Qubit",N)


10-element Vector{Index{Int64}}:
 (dim=2|id=829|"Qubit,Site,n=1")
 (dim=2|id=133|"Qubit,Site,n=2")
 (dim=2|id=116|"Qubit,Site,n=3")
 (dim=2|id=359|"Qubit,Site,n=4")
 (dim=2|id=781|"Qubit,Site,n=5")
 (dim=2|id=111|"Qubit,Site,n=6")
 (dim=2|id=530|"Qubit,Site,n=7")
 (dim=2|id=22|"Qubit,Site,n=8")
 (dim=2|id=150|"Qubit,Site,n=9")
 (dim=2|id=166|"Qubit,Site,n=10")

In [2]:
# Taking the input 
ITensors.disable_warn_order()
# @set_warn_order 18  # Sets the warning threshold to 18

x = range(0, stop=2π, length=2^N)
input_array = cos.(x)

array = input_array / norm(input_array) # Input

cutoff1 = 1E-18
maxdim1 = 10
# T = ITensor(array,s)
ψ = MPS(array,s;cutoff=cutoff1,maxdim=maxdim1)

println(maxlinkdim(ψ))

2


In [3]:
gates = ITensor[]

function H_gate(x)
    Hadamard = op("H",s[x]) # op  is  a function which converts matrices to ITensors
    
end

function swap_gate(i,j)
    swap = [ 1 0 0 0 ; 0 0 1 0; 0 1 0 0; 0 0 0 1] 
    swap_gate = op(swap, s[i] , s[j])
    
end


function phase_gate1(i,j,θ)
    phase = [ 1 0 0 0 ; 0 1 0 0; 0 0 1 0; 0 0 0 exp(-im * θ)] 
    phase_gate = op(phase, s[i] , s[j])
    # s[i] and s[j] are the control qubit 
    
end

phase_gate1 (generic function with 1 method)

In [4]:
# Define the QFT MPO
function qft_mpo(N::Int, sites)
    # Initialize MPO with sites
    W = MPO(sites,"I")  
    
    cutoff=1e-15
    # Apply Hadamard and phase gates to build the QFT MPO
    
    
    for i in 1:N
        W = apply(op("H",sites[N+1-i]),W,cutoff=cutoff)

        for j in i+1:N
            θ = π / 2^(j-i)
            P = phase_gate1(N+1-i,N+1-j,θ)
            # Update MPO by combining with phase gate tensor
            W = apply(P,W,cutoff=cutoff)


        end
            
    end
    
    return W
end

# Generate the QFT MPO
W = qft_mpo(N, s)
println(maxlinkdim(W))

orthogonalize!(W, 1)   # Orthogonalize W at the first index
orthogonalize!(ψ, 1) # Orthogonalize Psi at the first index

start = time()
# Contract MPO and MPS
result = contract(W, ψ)

end_time = time()

9


1.746188075637429e9

In [5]:
function generate_basis_states(n)
    basis_states = []
    for i in 0:(2^n - 1)
        binary_str = bitstring(i)[end-n+1:end]  # Get last n bits of bitstring
        push!(basis_states, collect(binary_str))  # Collect splits string into array of characters
    end
    return basis_states
end

MPS_states = []
basis_states = generate_basis_states(N)

for i in 1:(2^N)
    push!(MPS_states, MPS(s,string.(basis_states[i])))
end



In [11]:
N1 = 5
for i in 1:(2^N1)
    print(inner(MPS_states[i], result)*norm(input_array)*2^(N/2))
    println()
end



1.00000000000559 + 4.632293405122696e-13im
512.2469078046568 + 1.571558761146402im
-0.3341948409138523 - 0.00205061959737133im
-0.1252680535618643 - 0.0011529852805558477im
-0.06679907029690221 - 0.0008197890775532405im
-0.04174501943560903 - 0.0006404108107409514im
-0.028622446958635413 - 0.0005269349230143842im
-0.020868515544007454 - 0.00044823553302179885im
-0.015897255792385694 - 0.0003902556122608021im
-0.012518391464215543 - 0.0003457413638457077im
-0.010117618841217918 - 0.0003105021867398248im
-0.00834371774300204 - 0.0002816876597465609im
-0.006998297193856162 - 0.00025776294368294903im
-0.005955444612446044 - 0.00023764826546167502im
-0.005134041565865841 - 0.00022065117929898984im
-0.004466729179414824 - 0.00020570153720518355im
-0.003926202663654118 - 0.00019288234866366715im
-0.003472751736351817 - 0.00018128697898088736im
-0.0030803923838855474 - 0.00017028201792981465im
-0.002776699608728653 - 0.00016203993953959724im
-0.0025175139973375865 - 0.00015466553446077854im
-0

In [7]:
println("Time taken: ", end_time - start, " seconds")

Time taken: 3.4648709297180176 seconds


In [8]:
using FFTW

start = time()
x = fft(input_array)
ent = time()

for a in x
    println(a)
    println()
end

0.9999999999999611 + 0.0im

512.2469078046561 + 1.5715587611624355im

-0.3341948409195723 - 0.0020506195965590713im

-0.12526805356525644 - 0.001152985282242286im

-0.06679907029818426 - 0.0008197890772241939im

-0.04174501943544935 - 0.000640410810448538im

-0.02862244696224466 - 0.0005269349228454898im

-0.020868515626991057 - 0.00044823554052797563im

-0.01589817032197447 - 0.0003902781754583735im

-0.012518390647178822 - 0.0003457413427313978im

-0.010114617310120705 - 0.0003104099686938256im

-0.008343430380759435 - 0.0002816775041956714im

-0.007000450695605336 - 0.00025784186706706043im

-0.005957769502909865 - 0.00023774277487236044im

-0.005131969053112915 - 0.00022056122327880634im

-0.004466742892233909 - 0.00020570216508650077im

-0.003922958939300011 - 0.0001927226144626203im

-0.003472730131926817 - 0.00018128585087564853im

-0.0030957449408861373 - 0.00017113127429543658im

-0.0027769236562410326 - 0.00016205399429287704im

-0.0025048795974308485 - 0.00015389066612252845

In [9]:
println("Time taken: ", ent - start, " seconds")

Time taken: 0.0023238658905029297 seconds


In [10]:
# # Define the QFT MPO
# function qft_mpo(N::Int, sites)
#     # Initialize MPO with sites
#     W = MPO(sites,"I")  
    
#     cutoff=1e-15
#     # Apply Hadamard and phase gates to build the QFT MPO
#     r_max = N
    
#     for i in 1:N
#         W = apply(op("H",sites[1]),W,cutoff=cutoff)

#         for j in 2:r_max
            
#             P = phase_gate(j,j-1,j)
#             swap = swap_gate(j-1,j)
#             # Update MPO by combining with phase gate tensor
#             W = apply(P,W,cutoff=cutoff)
#             W = apply(swap,W,cutoff=cutoff)

#         end
        
#     r_max = r_max - 1    
#     end
    
#     return W
# end

# # Generate the QFT MPO
# W = qft_mpo(N, s)
# println(maxlinkdim(W))
# start = time()
# # Contract MPO and MPS
# result = contract(W, ψ)

# end_time = time()